In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import FeatureHasher
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from scipy import sparse
from itertools import product

In [ ]:
# Читаем входные данные
transactions = pd.read_csv('../data/raw/transactions.csv')
customers_gender = pd.read_csv('../data/raw/customers_gender_train.csv')
cuses_test = list(set(transactions.customer_id.unique().tolist()).difference(customers_gender.customer_id.unique()))
all_cuses = transactions.customer_id.unique()
all_mcc = transactions.mcc_code.unique()

# Берём расходные транзакции и формируем тестовую выборку
transactions = transactions[transactions.amount < 0].copy()
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

# Сдвигаем нумерацию дней для того, чтобы окончание обучающих данных совпало с окончанием месяца
transactions.day += 29 - transactions['day'].max()%30

# Добавляем признаки, относящиеся к дате.
transactions['month_num'] = (transactions.day) // 30
transactions['year_num'] = (transactions.day) // 365

test_transactions = transactions[transactions.month_num == 15]
train_transactions = transactions[transactions.month_num < 15]

test_transactions = test_transactions.set_index('customer_id')
test_transactions = test_transactions.loc[cuses_test]
test_transactions = test_transactions.reset_index()

grid = list(product(*[all_cuses, all_mcc, range(10, 15)]))
train_grid = pd.DataFrame(grid, columns = ['customer_id', 'mcc_code', 'month_num'])

test_grid = list(product(*[cuses_test, all_mcc]))       
test_grid = pd.DataFrame(test_grid, columns = ['customer_id', 'mcc_code'])
test_grid['month_num'] = 15

test = pd.merge(test_grid,
         test_transactions.groupby(['year_num', 'month_num', 'customer_id', 'mcc_code'])[['amount']].sum().reset_index(),
         how='left').fillna(0)

train = pd.merge(train_grid,
         train_transactions.groupby(['year_num', 'month_num', 'customer_id', 'mcc_code'])[['amount']].sum().reset_index(),
         how='left').fillna(0)

for month_shift in range(1, 3):
    train_shift = train.copy()
    train_shift['month_num'] = train_shift['month_num'] + month_shift
    train_shift = train_shift.rename(columns={"amount" : 'amount_{0}'.format(month_shift)})  
    train_shift = train_shift[['year_num', 'month_num', 'customer_id', 'mcc_code', 'amount_{0}'.format(month_shift)]]

    train = pd.merge(train, train_shift, 
                                  on=['year_num', 'month_num', 'customer_id', 'mcc_code'], how='left').fillna(0)
    test = pd.merge(test, train_shift, 
                                 on=['year_num', 'month_num', 'customer_id', 'mcc_code'], how='left').fillna(0)

hasher = FeatureHasher(n_features=10000, input_type='string')
train_sparse = \
    hasher.fit_transform(train[['year_num', 'month_num', 'customer_id', 'mcc_code']].astype(str).as_matrix())

test_sparse = \
    hasher.transform(test[['year_num', 'month_num', 'customer_id', 'mcc_code']].astype(str).as_matrix())

train_sparse = sparse.hstack([train_sparse,
                              np.log(np.abs(train[['amount_1', 'amount_2']]) + 1).as_matrix()
                             ])

test_sparse = sparse.hstack([test_sparse,
                             np.log(np.abs(test[['amount_1', 'amount_2']]) + 1).as_matrix()
                            ])

In [ ]:
# Применяем StandardScalar

scaler = StandardScaler(with_mean=False)
scaler.fit(train_sparse)
train_sparse_scaled = scaler.fit_transform(train_sparse)
test_sparse_scaled = scaler.fit_transform(test_sparse)

In [ ]:
# Обучаемся на всех данных

shift = 1
clf = LinearRegression()
clf.fit(train_sparse_scaled, np.log(-train['amount'] + shift))

In [ ]:
# Предсказываем объём трат для тестовых данных и создаём файл с ответом

test['volume'] = np.e ** clf.predict(test_sparse_scaled) - shift
test[['customer_id', 'mcc_code', 'volume']].to_csv('../data/submits/task3.csv', index=False)